In [ ]:
import datetime
from datetime import date
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import requests 

In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import time
chrome_options = webdriver.ChromeOptions()

#chrome_options.add_argument("--headless")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('"C:\chrome_driver\chromedriver.exe"', chrome_options = chrome_options)
time.sleep(2)

In [ ]:
url = "https://dart.fss.or.kr/dsab007/main.do"
driver.get(url)
driver.implicitly_wait(10)
selectpath = Select(driver.find_element(By.XPATH, "//*[@id='option']"))
selectpath.select_by_index(2)
input = driver.find_element(By.XPATH, '//*[@id="reportName"]')
input.send_keys('신규시설투자등')
driver.find_element(By.XPATH, '//*[@id="date5"]').send_keys(Keys.ENTER)
driver.find_element(By.XPATH, '//*[@id="date5"]').send_keys(Keys.ENTER)
selectpath = Select(driver.find_element(By.XPATH, '//*[@id="maxResultsCb"]'))
selectpath.select_by_index(3)
input.send_keys(Keys.RETURN)
driver.implicitly_wait(10)
time.sleep(2)
#검색도착
conamelist = []
hreflist = []
for k in range(5):        
    #try:    
        for i in range(1,101):
            try:
                time.sleep(0.5)
                tds = driver.find_element(By.XPATH, f'//*[@id="tbody"]/tr[{i}]/td[3]')
                a = tds.find_element(By.TAG_NAME, 'a')
                href = a.get_attribute('href')
                coname = driver.find_element(By.XPATH, f'//*[@id="tbody"]/tr[{i}]/td[4]').get_attribute('innerText')
                hreflist.append(href)
                conamelist.append(coname)
                print(i)
                
            except:
                print('오류','-'*50)
        driver.find_element(By.XPATH, f'//*[@id="psWrap"]/div[2]/ul/li[{k+2}]/a').send_keys(Keys.ENTER)
        time.sleep(4)
   # except:
     #   break


In [ ]:
len(conamelist)

In [ ]:
conamelist
invlist = []
mymoneylist = []
driver.implicitly_wait(0.5)
mymoneyratlist = []
purposelist = []
for link,coname,number in zip(hreflist,conamelist,range(len(conamelist))):    
    driver.get(link)
    iframes = driver.find_element(By.TAG_NAME, 'iframe')
    driver.switch_to.frame(iframes)
    try: 
        inv = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[2]/td[3]/span').get_attribute('innerText')
        invlist.append(inv.replace(',',''))
        mymoney = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[3]/td[2]/span').get_attribute('innerText')
        mymoneylist.append(mymoney.replace(',', ''))
        mymoneyrat = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[4]/td[2]/span').get_attribute('innerText')
        mymoneylist.append(mymoneyrat)
        purpose = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[6]/td[2]/span').get_attribute('innerText')
        purposelist.append(purpose.strip())
        time.sleep(2)
    except:
        inv = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[3]/td[3]/span').get_attribute('innerText')
        invlist.append(inv.replace(',',''))
        mymoney = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[4]/td[2]/span').get_attribute('innerText')
        mymoneylist.append(mymoney.replace(',', ''))
        mymoneyrat = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[5]/td[2]/span').get_attribute('innerText')
        mymoneylist.append(mymoneyrat)
        purpose = driver.find_element(By.XPATH, '//*[@id="XFormD1_Form0_Table0"]/tbody/tr[7]/td[2]/span').get_attribute('innerText')
        purposelist.append(purpose.strip())
        time.sleep(2)
    print(f'{number}' + coname)
    print(mymoneyrat)


In [ ]:
realmymoneylist = mymoneylist[0::2]
mymoneyratlist = mymoneylist[1::2]

In [ ]:
df = pd.DataFrame({'기업명': conamelist ,
                   '투자금액' : invlist , 
                   '자기자본' : realmymoneylist, 
                   '자기자본비율' : mymoneyratlist, 
                   '투자목적' : purposelist})
df

In [ ]:
df['자기자본비율'].astype('str')